# Описание проекта

Необходимо было по данным о пользователях мобильной связи построить рекомендации одного из двух тарифов мобильной связи.

Рассмотрено несколько моделей с различными гиперпараметрами. Метрикой качества является *accuracy*

## Импорт библиотек

In [1]:
import pandas as pd                                              # импорт библиотеки pandas
import warnings                                                  # импорт службы warnings
warnings.simplefilter("ignore")                                  # игнорируем предупреждения
from sklearn.model_selection import train_test_split             # импорт функции разбиения выборки
from sklearn.tree import DecisionTreeClassifier                  # импорт модели решающего дерева
from sklearn.ensemble import RandomForestClassifier              # импорт модели случайного леса
from sklearn.linear_model import LogisticRegression              # импорт модели логистической регрессии
from sklearn.metrics import accuracy_score                       # импорт функции подсчёта среднеквадратичной ошибки

## Откройте и изучите файл

In [2]:
df = pd.read_csv('...')       # открываем файл
df.info()                                              # выводим информацию о датафрейме
df.head()                                              # смотрим, как выглядят первые 5 строчек датафрейма

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


**Вывод**

В датафрейме отсутствуют пропуски и все данные имеют нужный формат.

## Разбейте данные на выборки

In [3]:
df_train_valid, df_test = train_test_split(df, test_size=0.2, random_state=12345)        # разбиваем выборку на тестовую
# и оставшуюся
df_train, df_valid = train_test_split(df_train_valid, test_size=0.25, random_state=1234) # оставшуюся выборку разбиваем на 
# обучающую и валидационную
print('Размер обучающей выборки: ', df_train.shape[0])         # выводим размер обучающей выборки
print('Размер тестовой выборки: ', df_test.shape[0])           # выводим размер тестовой выборки
print('Размер валидационной выборки: ',df_valid.shape[0])      # выводим размер валидационной выборки

Размер обучающей выборки:  1928
Размер тестовой выборки:  643
Размер валидационной выборки:  643


**Вывод**

Поскольку тестовая выборка берётся из представленного датафрейма, то соотношение выборок обучающей, валидационной и тестовой выглядит так: 60% : 20% : 20% 

## Исследуйте модели

### Модель решающего дерева

In [4]:
features_train = df_train.drop('is_ultra', axis=1) # выделяем признаки в обучающей выборке
target_train = df_train['is_ultra']                # выделяем целевой признак в обучающей выборке

features_valid = df_valid.drop('is_ultra', axis=1) # выделяем признаки в валидационной выборке
target_valid = df_valid['is_ultra']                # выделяем целевой признак в валидационной выборке

best_model_tree = None                             #
best_result_tree = 0                               # создаём переменные для лучшей модели, лучшего значения accuracy
best_depth_tree = 0                                # лучшего значения глудины дерева

for depth_tree in range(1, 11):                    # создаём цикл по глубине дерева
    model_tree = DecisionTreeClassifier(random_state=123, max_depth=depth_tree) # создаём модель с изменяющейся глубиной дерева
    model_tree.fit(features_train, target_train) # обучаем модель на обучающей выборке
    predictions = model_tree.predict(features_valid) # получаем предсказания модели
    result_tree = accuracy_score(target_valid, predictions) # считаем качество модели
    if result_tree > best_result_tree:   # c помощью условия if         
        best_model_tree = model_tree     # определяем лучшую модель,
        best_result_tree = result_tree   # лучшее значение accuracy, 
        best_depth_tree = depth_tree     # лучшую глубину дерева
print("Accuracy лучшей модели на валидационной выборке:", best_result_tree)
print("Глубина дерева лучшей модели на валидационной выборке:", best_depth_tree)

Accuracy лучшей модели на валидационной выборке: 0.8055987558320373
Глубина дерева лучшей модели на валидационной выборке: 3


**Вывод**

Значение accuracy превышает контрольное значение 0,75. Посмотрим результат на тестовой выборке.

### Модель случайного леса

In [5]:
best_model_forest = None  # создаём переменные для лучшей модели, 
best_result_forest = 0    # лучшего значения accuracy
best_est = 0              # лучшего количества деревьев
best_depth_forest = 0     # лучшей глубины деревьев
best_samples_split = 0    # лучшего значения минимального кол-ва выборок для разделения узла
for samples_split in range(2, 7):         # цикл по минимальному кол-ву выборок для разделения узла
    for est in range(1, 11):              # цикл по количеству деревьев
        for depth_forest in range(1, 6):  # цикл по глубине деревьев
            model_forest = RandomForestClassifier(
                random_state=1234, n_estimators=est, max_depth = depth_forest, min_samples_split=samples_split) # создаём 
# модель с изменяемым количеством деревьев, глубиной деревьев, минимальное кол-во выборок для разделения узла 
            model_forest.fit(features_train, target_train) # обучаем модель на обучающей выборке
            result_forest = model_forest.score(features_valid, target_valid) # считаем качество модели на валидационной выборке
            if result_forest > best_result_forest: # c помощью условия if
                best_model_forest = model_forest   # сохраняем наилучшую модель
                best_result_forest = result_forest # сохраняем наилучшее значение метрики accuracy на валидационных данных
                best_est = est                     # сохраняем наилучшее количество деревьев
                best_depth_forest = depth_forest   # сохраняем наилучшую глубину деревьев
                best_samples_split = samples_split # сохраняем наилучшее минимальное кол-во выборок для разделения узла 
print("Accuracy наилучшей модели на валидационной выборке:", best_result_forest)
print("Оптимальное количество деревьев модели на валидационной выборке:", best_est)
print("Оптимальная глубина деревьев модели на валидационной выборке:", best_depth_forest)
print("Оптимальное минимальное кол-во выборок для разделения узла на валидационной выборке:", best_samples_split)

Accuracy наилучшей модели на валидационной выборке: 0.8164852255054432
Оптимальное количество деревьев модели на валидационной выборке: 6
Оптимальная глубина деревьев модели на валидационной выборке: 5
Оптимальное минимальное кол-во выборок для разделения узла на валидационной выборке: 3


**Вывод**

В ходе построения модели случайного леса с варьированием гиперпараметров было обнаружено, что изменение одного гиперпараметра приводит к изменению оптимальных значений других гиперпараметров. Так, например, если не менять гиперпараметр min_samples_split, то оптимальное кол-во деревьев будет 4, а глубина 3. Если же провести варьирование гиперпараметра min_samples_split, то, как видно, оптимальные параметры стали другими.

Полученное значение accuracy на валидационной выборке для модели случайного леса превышает контрольный уровень, оно получилось больше на 1%, чем для модели решающего дерева.

### Модель логистической регрессии

In [6]:
model_regression = LogisticRegression(random_state=12345) # инициализируем модель логистической регрессии 
# с параметром random_state=12345
model_regression.fit(features_train, target_train) # обучаем модель на тренировочной выборке
result_regression = model_regression.score(features_valid, target_valid) # считаем качество модели на валидационной выборке
print("Accuracy модели логистической регрессии на валидационной выборке:", result_regression)

Accuracy модели логистической регрессии на валидационной выборке: 0.7511664074650077


**Вывод**

Значение accuracy превышает контрольную величину, но всего на 0,1%.

## Проверьте модель на тестовой выборке

In [7]:
features_test = df_test.drop('is_ultra', axis=1) # выделяем признаки тестовой выборки
target_test = df_test['is_ultra']                # выделяем целевой признак тестовой выборки

prediction_test_tree = best_model_tree.predict(features_test)   # строим предсказания по модели решающего дерева
result_test_tree = accuracy_score(target_test, prediction_test_tree) # считаем качество модели на тестовой выборке
print("Accuracy лучшей модели решающего дерева на тестовой выборке:", result_test_tree)

result_test_forest = best_model_forest.score(features_test, target_test) # применяем модель случайного дерева к тестовой выборке  
print("Accuracy лучшей модели случайного леса на тестовой выборке:", result_test_forest)

result_test_regression = model_regression.score(features_test, target_test) # применяем модель логистической регрессии к 
# тестовой выборке
print("Accuracy модели логистической регрессии на тестовой выборке:", result_test_regression)

Accuracy лучшей модели решающего дерева на тестовой выборке: 0.7853810264385692
Accuracy лучшей модели случайного леса на тестовой выборке: 0.7931570762052877
Accuracy модели логистической регрессии на тестовой выборке: 0.7573872472783826


**Вывод**

Значение accuracy на всех трёх моделях превышает 0.75. Самой точной моделью на тестовой выборке оказалась модель случайного леса с 6 деревьями, глубиной деревьев - 5 и минимальным кол-вом выборок для разделения узла - 3.

## (бонус) Проверьте модели на адекватность

Я бы проверил наши модели и посмотрел точность предсказаний на обучающей выборке:

In [8]:
prediction_train_tree = best_model_tree.predict(features_train) # строим предсказания модели решающего дерева на обучающей
# выборке
result_train_tree = accuracy_score(target_train, prediction_train_tree) # считаем качество модели
print("Accuracy лучшей модели решающего дерева на обучающей выборке:", result_train_tree)

result_train_forest = best_model_forest.score(features_train, target_train) # применяем модели случайного леса к обучающей
# выборке и смотрим результат
print("Accuracy лучшей модели случайного леса на обучающей выборке:", result_train_forest)

result_train_regression = model_regression.score(features_train, target_train) # применяем модель логистической регрессии
# к обучающей выборке и смотрим результат
print("Accuracy модели логистической регрессии на обучающей выборке:", result_train_regression)

Accuracy лучшей модели решающего дерева на обучающей выборке: 0.799792531120332
Accuracy лучшей модели случайного леса на обучающей выборке: 0.8132780082987552
Accuracy модели логистической регрессии на обучающей выборке: 0.7468879668049793


**Вывод**

Модели решающего дерева и случайного леса готовились усердно и показывают более высокую точность на обучающей выборке, чем на тестовой на 1,5-2%. Логистическая регрессия, наоборот, при подготовке к экзамену усвоила материал на 1% хуже, чем ответила на самом экзамене. Вцелом все 3 модели пока выглядят адекватно. 

Следующим этапом я бы проверил наши модели на устойчивость: добавил бы в данные обучающей выборки строчку с какой-нибудь абракадаброй. Если система устойчива, то такое возмущение не должно сильно сказываться на точности предсказаний.

**Модель решающего дерева**

In [9]:
target_train_2 =  target_train.copy() # создаём копию целевого признака обучающей выборки, наличие цифры 2 в переменных 
# будет означать, что эта переменная связана с новой обучающей выборкой, содержащей особенности
target_train_2[6666] = 34566          # добавляем одно значение в новый столбец целевого признака,всего 1 или 0
# такое число будет явно не ожиданным, если диапазон чисел 
features_train_2 = features_train.copy() # создаём копию  признаков обучающей выборки
new_row = pd.Series(data={'calls':6677, 'minutes':'12', 'messages':998, 'mb_used':2}, name=6666) # создаём строку новых 
# признаков
features_train_2 = features_train_2.append(new_row, ignore_index=False) # добавляем эту строку к новой выборке

best_model_tree_2 = None # создаём переменные для лучшей модели, 
best_result_tree_2 = 0   # для лучшего значения accuracy
best_depth_tree_2 = 0    # для лучшего значения глубины дерева
for depth_tree_2 in range(1, 11):  # запускаем цикл по глубине дерева
    model_tree_2 = DecisionTreeClassifier(random_state=123, max_depth=depth_tree_2) # создаём модель решающего дерева
# с изменяемой глубиной дерева
    model_tree_2.fit(features_train_2, target_train_2) # обучаем модель на новой обучающей выборке
    predictions_2 = model_tree_2.predict(features_valid) # получаем предсказания модели
    result_tree_2 = accuracy_score(target_valid, predictions_2) # считаем качество новой модели
    if result_tree_2 > best_result_tree_2:             # c помощью условия if        
        best_model_tree_2 = model_tree_2               # определяем лучшую модель,
        best_result_tree_2 = result_tree_2             # лучшее значение accuracy,
        best_depth_tree_2 = depth_tree_2               # лучшую глубину дерева
print("Accuracy модели решающего дерева на валидационной выборке:", best_result_tree_2)
print("Глубина дерева на валидационной выборке:", best_depth_tree_2)  

Accuracy модели решающего дерева на валидационной выборке: 0.8055987558320373
Глубина дерева на валидационной выборке: 3


**Вывод**

Модель решающего дерева не шелохнуласть от попыток его раскачать неадекватной строчкой в датафрейме.

**Модель случайного леса**

In [10]:
best_model_forest_2 = None # создаём переменные для лучшей модели, 
best_result_forest_2 = 0   # лучшего значения accuracy
best_est_2 = 0             # лучшего количества деревьев
best_depth_forest_2 = 0    # лучшей глубины деревьев
best_samples_split_2 = 0   # лучшего значения минимального кол-ва выборок для разделения узла
for samples_split_2 in range(2, 7):         # цикл по минимальному кол-ву выборок для разделения узла
    for est_2 in range(1, 11):              # цикл по количеству деревьев
        for depth_forest_2 in range(1, 6):  # цикл по глубине деревьев
            model_forest_2 = RandomForestClassifier(
                random_state=1234, n_estimators=est_2, max_depth = depth_forest_2, min_samples_split=samples_split_2) # создаём 
# модель с изменяемым количеством деревьев, глубиной деревьев, минимальное кол-во выборок для разделения узла 
            model_forest_2.fit(features_train_2, target_train_2) # обучаем модель на новой обучающей выборке
            result_forest_2 = model_forest_2.score(features_valid, target_valid) # считаем качество новой модели на 
# валидационной выборке
            if result_forest_2 > best_result_forest_2:  # c помощью условия if
                best_model_forest_2 = model_forest_2    # сохраняем наилучшую модель
                best_result_forest_2 = result_forest_2  # сохраняем наилучшее значение метрики accuracy на 
# валидационных данных
                best_est_2 = est_2                      # сохраняем наилучшее количество деревьев
                best_depth_forest_2 = depth_forest_2    # сохраняем наилучшую глубину деревьев
                best_samples_split_2 = samples_split_2  # сохраняем наилучшее минимальное кол-во выборок для разделения узла 
print("Accuracy наилучшей модели на валидационной выборке:", best_result_forest_2)
print("Оптимальное количество деревьев модели на валидационной выборке:", best_est_2)
print("Оптимальная глубина деревьев модели на валидационной выборке:", best_depth_forest_2)
print("Оптимальное минимальное кол-во выборок для разделения узла на валидационной выборке:", best_samples_split_2)

Accuracy наилучшей модели на валидационной выборке: 0.8180404354587869
Оптимальное количество деревьев модели на валидационной выборке: 4
Оптимальная глубина деревьев модели на валидационной выборке: 5
Оптимальное минимальное кол-во выборок для разделения узла на валидационной выборке: 5


**Вывод**

Модель случайного леса демонстрирует изменения оптимальных гиперпараметров при добавлении некорректной строчки в обучающую выборку. Величина accuracy даже увеличилась. Проверим accuracy при старых гиперпараметрах. 

In [11]:

model_forest_2 = RandomForestClassifier(
    random_state=1234, n_estimators=6, max_depth = 5, min_samples_split=3) # обучаем модель с фиксированными значениями 
# количества деревьев, их глубины и минимальным кол-вом выборок для разделения узла, как в главе исследования модели
# случайного леса
model_forest_2.fit(features_train_2, target_train_2) # обучаем модель на тренировочной выборке с особенностью
result_forest_2 = model_forest_2.score(features_valid, target_valid) # считаем качество модели на валидационной выборке

print("Accuracy модели на валидационной выборке при фиксации гиперпараметров:", result_forest_2)

Accuracy модели на валидационной выборке при фиксации гиперпараметров: 0.8133748055987559


**Вывод**

Если принудительно поставить величины гипермараметров такими, какими они были в главе исследования модели случайного леса, то accuracy изменится на 0,5%. 

**Модель логисической регрессии**

In [12]:
model_regression_2 = LogisticRegression(random_state=12345) # инициализируем модель логистической регрессии 
#с параметром random_state=12345
model_regression_2.fit(features_train_2, target_train_2) # обучаем модель на тренировочной выборке с особенностью
result_regression_2 = model_regression_2.score(features_valid, target_valid) # считаем качество модели на валидационной выборке

print("Accuracy модели логистической регрессии на валидационной выборке:", result_regression_2)

Accuracy модели логистической регрессии на валидационной выборке: 0.7200622083981337


**Вывод**

А вот и существенная особенность - модель логистической регрессии оказалась чувствительной к выбросам в обучающей выборке. 1 строчка понизила точность более чем на 3%.

### Общий вывод

В ходе построения оптимальной модели решения задачи классификации пришлось работать с датафреймом, избавленным от пропусков и неверных типов данных. 

Датафрейм был поделён на три выборки обучающую (60%), валидационную (20%), тестовую(20%).

Рассматривались три типа моделей:
- решающее дерево
- случайный лес
- логистическая регрессия

Для модели решающего дерева оптимальный гиперпараметр глубины составил 3.
Для модели случайного леса оптимальные гиперпараметры равны: количество деревьев - 6, глубина деревьев - 5, минимальное кол-во выборок для разделения узла - 3.

На тестовой выборке модели показали следующие значения accuracy:
- решающее дерево  0.785
- случайный лес  0.793
- логистическая регрессия  0.757

Победителем стал случайный лес.

Что же касается проверки на адекватность, то я пытался раскачать модели внесением одной некорректной строчки в обучающую выборку. В итоге, получилось, что модель решающего дерева не поменяла точность предсказания и глубину дерева. Случайный лес поменял оптимальные гиперпараметры, поменял точность предсказания, но не так сильно, как модель логистической регрессии. Логистическая регрессия потеряла в точность более 3 %. Вывод: если в данных не уверен на 100% логистическую регрессию применять не стоит - предсказания могут стать неадекватными, лучше воспользоваться решающим деревом.  



## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гиперпараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
